<a href="https://colab.research.google.com/github/Anuargharsh/Movie/blob/main/Makeover2MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#read in all the words
words = open('/content/names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [12]:
enumerate(chars)
print(list(enumerate(chars)))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g'), (7, 'h'), (8, 'i'), (9, 'j'), (10, 'k'), (11, 'l'), (12, 'm'), (13, 'n'), (14, 'o'), (15, 'p'), (16, 'q'), (17, 'r'), (18, 's'), (19, 't'), (20, 'u'), (21, 'v'), (22, 'w'), (23, 'x'), (24, 'y'), (25, 'z')]


In [16]:
print(stoi.items())
print(list(stoi.items()))

dict_items([('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5), ('f', 6), ('g', 7), ('h', 8), ('i', 9), ('j', 10), ('k', 11), ('l', 12), ('m', 13), ('n', 14), ('o', 15), ('p', 16), ('q', 17), ('r', 18), ('s', 19), ('t', 20), ('u', 21), ('v', 22), ('w', 23), ('x', 24), ('y', 25), ('z', 26), ('.', 0)])
[('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5), ('f', 6), ('g', 7), ('h', 8), ('i', 9), ('j', 10), ('k', 11), ('l', 12), ('m', 13), ('n', 14), ('o', 15), ('p', 16), ('q', 17), ('r', 18), ('s', 19), ('t', 20), ('u', 21), ('v', 22), ('w', 23), ('x', 24), ('y', 25), ('z', 26), ('.', 0)]


In [29]:
#build the vocabulary of characters and mapping

block_size = 3 #context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [30]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [42]:
C = torch.randn((27,2))

In [43]:
C[5]

tensor([ 1.2358, -0.2458])

In [53]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [54]:
w1 = torch.randn((6,100))
b1 = torch.randn(100)

In [85]:
h = torch.tanh(emb.view(-1,6) @ w1 + b1)

In [86]:
h

tensor([[-0.0159,  0.9994,  0.9985,  ..., -0.7831, -0.4773, -0.9687],
        [ 0.0497,  0.6316,  0.9862,  ...,  0.3417, -0.9388, -0.1663],
        [-0.0388,  0.9956,  1.0000,  ..., -0.1568,  0.8656, -0.8907],
        ...,
        [-0.9998,  0.9409,  0.9316,  ..., -0.9910, -0.9759,  0.8948],
        [-0.3206, -0.3414,  0.7984,  ..., -0.5007, -0.8297,  0.5883],
        [-0.9166,  0.9763,  0.9999,  ..., -0.5745, -0.2683, -0.5527]])

In [87]:
h.shape

torch.Size([32, 100])

In [88]:
w2 = torch.randn((100,27))
b2 = torch.randn(27)

In [89]:
logits = h @ w2 + b2

In [90]:
logits.shape

torch.Size([32, 27])

In [91]:
logits

tensor([[ 1.1554e+00, -1.7909e-01, -1.4372e+00,  2.1412e+00, -1.4703e+00,
          4.5249e+00, -2.1966e+00, -4.1568e+00, -8.8706e-01, -2.3766e+00,
          1.0037e+01, -5.9048e+00,  1.0720e+01,  1.2515e+01,  5.5559e+00,
          7.5206e+00,  4.2145e+00, -1.2563e+01, -4.6380e+00,  9.7909e+00,
          1.5877e+00, -7.5100e+00, -7.5377e+00,  2.8622e+00,  9.7893e+00,
         -8.5290e+00, -1.4052e+01],
        [-9.3651e+00,  8.2198e+00,  1.4788e+01, -2.0262e+00, -8.3519e+00,
          9.9650e-02,  3.0264e+00,  9.0480e+00,  6.7273e+00,  7.7275e+00,
          2.0064e+01, -6.6413e+00, -9.4236e-01,  5.1318e-01,  4.0097e+00,
          5.2099e+00,  8.2119e+00, -1.3454e+01, -3.6958e+00,  9.5094e+00,
         -8.2160e+00,  9.2911e+00,  2.2547e+00,  1.6355e+01,  4.3476e+00,
         -1.0339e+01, -4.3928e-02],
        [-9.7905e+00,  1.0906e+01, -3.2298e+00, -2.5597e+00, -9.2900e+00,
          5.4212e+00,  6.8189e+00,  8.0933e+00, -1.0738e+00,  1.6091e+01,
          1.5443e+01, -8.9698e+00, -5.46

In [93]:
counts = logits.exp()

In [94]:
prob = counts / counts.sum(1, keepdims = True)

In [95]:
prob.shape

torch.Size([32, 27])

In [101]:
loss = -prob[torch.arange(32),Y].log().mean()
loss

tensor(15.9991)

#Now made respectable


In [102]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [105]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn((100), generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn((27), generator = g)
parameters = [C,W1,b1,W2,b2]

In [106]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [107]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdim = True)
loss = - prob[torch.arange(32),Y].log().mean()
loss

tensor(17.7697)